## Preprocessing

In [16]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [17]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)

In [18]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [19]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()


T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [20]:
# Choose a cutoff value and create a list of application types to be replaced with less than 3 value counts.
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < 500].index.to_list()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [21]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1820        1
C3700        1
C2170        1
C6100        1
C2380        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [22]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df["CLASSIFICATION"].value_counts()[application_df["CLASSIFICATION"].value_counts() > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C1720        6
C2400        6
C4100        6
C1600        5
C1257        5
C1260        3
C2710        3
C0           3
C1267        2
C3200        2
C1234        2
C1256        2
C1246        2
Name: CLASSIFICATION, dtype: int64

In [23]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df["CLASSIFICATION"].value_counts()[application_df["CLASSIFICATION"].value_counts() < 300].index.to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [24]:
# Convert categorical data to numeric with `pd.get_dummies`
df = pd.get_dummies(application_df)

In [40]:
# Split our preprocessed data into our features and target arrays
X = df.drop("IS_SUCCESSFUL", axis=1)
y = df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)


In [41]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [42]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=3, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 5)                 225       
_________________________________________________________________
dense_13 (Dense)             (None, 3)                 18        
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 4         
Total params: 247
Trainable params: 247
Non-trainable params: 0
_________________________________________________________________


In [43]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

In [44]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Train on 25724 samples
Epoch 1/50
25724/25724 [==============================] - 2s 80us/sample - loss: 0.6688 - accuracy: 0.5859
Epoch 2/50
25724/25724 [==============================] - 1s 50us/sample - loss: 0.6106 - accuracy: 0.6978
Epoch 3/50
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5977 - accuracy: 0.7017
Epoch 4/50
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5924 - accuracy: 0.7044
Epoch 5/50
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5863 - accuracy: 0.7107
Epoch 6/50
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5737 - accuracy: 0.7229
Epoch 7/50
25724/25724 [==============================] - 1s 54us/sample - loss: 0.5709 - accuracy: 0.7238
Epoch 8/50
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5693 - accuracy: 0.7236
Epoch 9/50
25724/25724 [==============================] - 1s 53us/sample - loss: 0.5671 - accuracy: 0.7267
Epoch 10/50
25

In [45]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 0s - loss: 0.5498 - accuracy: 0.7249
Loss: 0.5590202044259007, Accuracy: 0.7248979806900024


In [49]:
# Export our model to HDF5 file
nn_model.save("./Resources/alphabetSoupCharity_Optimization.h5")

In [51]:
# import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
model = load_model('./Resources/alphabetSoupCharity_Optimization.h5')

ModuleNotFoundError: No module named 'keras'